<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_FvS_QoQ_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 10.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100
#stock_ids = ['TCS', 'MOTILALOFS']

In [3]:
# change this
curr_quarter = 'Dec 2022'
prev_quarter = 'Sep 2022'
len(stock_ids)

75

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df_analysis = pd.DataFrame()

for stock_name in stock_ids: 

#stock_name = 'tcs'

    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"
    
    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')
        
        test_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")
            
        return test_list
    
    test_list = get_list(actual_link)
        
    print(len(test_list))
    n = len(test_list) 
    
    if n < 114:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        print(len(test_list))
    
    q = 0
        
    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   
        
    else:
        p = 6

    df_qreults = pd.DataFrame()
    
    if 'Gross NPA %'  in test_list:
        endkey = 'Gross NPA %'
    else:
        endkey = 'Raw PDF'

    if curr_quarter in test_list:
        quarter = curr_quarter
    else:
        quarter = prev_quarter

    #df_qreults['Quarter'] = test_list[test_list.index('Sep 2021')-p:test_list.index('Sep 2021')+1]
    #df_qreults['EPS(Q)'] = test_list[test_list.index('Raw PDF')-p-q-1:test_list.index('Raw PDF')-q]
    s = test_list.index(endkey)  - test_list.index('EPS in Rs')-2
    df_qreults['Quarter'] = test_list[test_list.index(quarter)-s:test_list.index(quarter)+1]    
    df_qreults['EPS(Q)'] = test_list[test_list.index('EPS in Rs')+1:test_list.index(endkey)]

    try:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Expenses\xa0+')-s-1:test_list.index('Expenses\xa0+')]
    except:
        df_qreults['Sales(Q)'] = test_list[test_list.index('Interest')-s-1:test_list.index('Interest')] 
    
    
    df_qreults['Sales(Q)']  = df_qreults['Sales(Q)'].str.replace(",", "")

    df_qreults['EPS(Q)'] = df_qreults['EPS(Q)'].astype(float).fillna(0.0)
    df_qreults['Sales(Q)'] = df_qreults['Sales(Q)'].astype(float).fillna(0.0)

    from pandas.tseries.offsets import MonthEnd
    import datetime

    df_qreults['Quarter'] = pd.to_datetime(df_qreults['Quarter'], format="%b %Y") + MonthEnd(1) + datetime.timedelta(days=46)
    df_qreults

    df_qreults['EPS(TTM)'] = df_qreults['EPS(Q)'] + df_qreults['EPS(Q)'].shift(1) + df_qreults['EPS(Q)'].shift(2) + \
                                df_qreults['EPS(Q)'].shift(3)
    
    df_qreults['Sales(TTM)'] = df_qreults['Sales(Q)'] + df_qreults['Sales(Q)'].shift(1) + df_qreults['Sales(Q)'].shift(2) + \
                                df_qreults['Sales(Q)'].shift(3)

    import pandas_datareader.data as web
    import yfinance as yfin
    yfin.pdr_override()

    df_stock = pd.DataFrame()

    for date in df_qreults['Quarter']:
        print(date)

        end_date = date
        start_date = (date + datetime.timedelta(days=-45)).strftime('%Y-%m-%d')


        stock_id = stock_name.upper() + '.NS'
        tmp = pd.DataFrame(web.DataReader(tickers = stock_id, start = start_date, end = end_date))
        tmp = tmp.tail(1)
        tmp['Stock'] = stock_id
        tmp['Quarter'] = end_date  
        tmp['Close'] = round(tmp['Close'],2)
        tmp.reset_index(inplace = True)
        cols = ['Stock','Quarter','Close']
        tmp =tmp[cols]    
        df_stock = df_stock.append(tmp)

    df_stock_analysis = df_stock.merge(df_qreults, on='Quarter').tail(6)
    df_stock_analysis['PE(TTM)'] = round(df_stock_analysis['Close']/df_stock_analysis['EPS(TTM)'],2)
    
    df_analysis = df_analysis.append(df_stock_analysis)
    
df_analysis

ASIANPAINT
172
2020-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2020-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2021-11-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-02-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-05-16 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-08-15 00:00:00
[*********************100%***********************]  1 of 1 completed
2022-1

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM)
7,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04
8,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44
9,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96
10,ASIANPAINT.NS,2022-08-15,3427.85,10.60,8607.0,36.27,32123.0,94.51
11,ASIANPAINT.NS,2022-11-15,3053.20,8.16,8458.0,38.22,33485.0,79.88
...,...,...,...,...,...,...,...,...
8,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91
9,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63
10,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83
11,ZEEL.NS,2022-08-15,241.80,1.11,1846.0,8.92,8261.0,27.11


In [5]:
df_analysis_final = pd.DataFrame()
for stock_id in df_analysis['Stock'].unique():
    tmp = df_analysis[df_analysis['Stock'] == stock_id]
    tmp['Prev_EPS(Q)'] = tmp['EPS(Q)'].shift(1)
    tmp['Prev_EPS(TTM)'] = tmp['EPS(TTM)'].shift(1)
    tmp['Prev_Sales'] = tmp['Sales(Q)'].shift(1)
    tmp['Prev_PE'] = tmp['PE(TTM)'].shift(1)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp['FR%'] = round((tmp['EPS(TTM)'] - tmp['Prev_EPS(TTM)'])*100/tmp['Prev_EPS(TTM)'],2)
    tmp['SR%'] = round(tmp['EPS(TTM)']*(tmp['PE(TTM)'] - tmp['Prev_PE'])*100/tmp['Prev_Close'],2)
    tmp['TR%'] = tmp['FR%'] + tmp['SR%']
    tmp['Sales%'] = round((tmp['Sales(Q)'] - tmp['Prev_Sales'])*100/tmp['Prev_Sales'],2)
    tmp['EPS%'] = round((tmp['EPS(Q)'] - tmp['Prev_EPS(Q)'])*100/tmp['Prev_EPS(Q)'],2)
    tmp['Return%'] = tmp['FR%'] + tmp['SR%']
    df_analysis_final = df_analysis_final.append(tmp)
    
df_analysis_final

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
7,ASIANPAINT.NS,2021-11-15,3122.90,6.21,7096.0,33.93,26120.0,92.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44,6.21,33.93,7096.0,92.04,3122.90,-6.84,7.49,0.65,20.17,70.53,0.65
9,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96,10.59,31.61,8527.0,99.44,3143.45,-0.03,-2.49,-2.52,-7.44,-16.24,-2.52
10,ASIANPAINT.NS,2022-08-15,3427.85,10.60,8607.0,36.27,32123.0,94.51,8.87,31.60,7893.0,96.96,3064.00,14.78,-2.90,11.88,9.05,19.50,11.88
11,ASIANPAINT.NS,2022-11-15,3053.20,8.16,8458.0,38.22,33485.0,79.88,10.60,36.27,8607.0,94.51,3427.85,5.38,-16.31,-10.93,-1.73,-23.02,-10.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91,2.23,10.24,1775.0,17.90,183.25,17.87,52.76,70.63,11.49,26.01,70.63
9,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63,2.81,12.07,1979.0,25.91,312.70,-8.70,-11.56,-20.26,6.77,10.68,-20.26
10,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83,3.11,11.02,2113.0,22.63,249.35,-8.89,4.83,-4.06,9.94,-39.23,-4.06
11,ZEEL.NS,2022-08-15,241.80,1.11,1846.0,8.92,8261.0,27.11,1.89,10.04,2323.0,23.83,239.25,-11.16,12.23,1.07,-20.53,-41.27,1.07


In [6]:
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
df_growth = df_analysis_final[cols]
df_growth[df_growth['Quarter'] == max(df_growth['Quarter'])].sort_values(by='Sales%', ascending=False).head(20)

,Stock,Quarter,Sales%,EPS%,Return%
12,PVR.NS,2023-02-15,36.97,-122.66,-6.38
12,TITAN.NS,2023-02-15,26.69,8.76,-6.13
12,EMAMILTD.NS,2023-02-15,20.76,29.02,-9.78
12,INDIGO.NS,2023-02-15,19.48,-189.85,15.10
12,BANKBARODA.NS,2023-02-15,14.49,26.60,3.32
12,IRCTC.NS,2023-02-15,13.90,12.72,-15.11
12,VOLTAS.NS,2023-02-15,13.46,1418.18,3.27
12,HDFC.NS,2023-02-15,13.35,1.33,1.02
12,BAJFINANCE.NS,2023-02-15,12.79,6.92,-8.41
12,KOTAKBANK.NS,2023-02-15,12.74,10.73,-7.81


In [7]:
from google.colab import drive
drive.mount('drive')

df_analysis_final.to_csv('/content/drive/My Drive/Nifty100_FvS_QoQ_Analysis.csv', index=False)

Mounted at drive


In [8]:
df_growth = pd.read_csv('/content/drive/My Drive/Nifty100_FvS_QoQ_Analysis.csv')
cols = ['Stock','Quarter','Sales%','EPS%','Return%']
#df_growth = df_growth[cols]
df_growth = df_growth.dropna()
df_growth

,Stock,Quarter,Close,EPS(Q),Sales(Q),EPS(TTM),Sales(TTM),PE(TTM),Prev_EPS(Q),Prev_EPS(TTM),Prev_Sales,Prev_PE,Prev_Close,FR%,SR%,TR%,Sales%,EPS%,Return%
1,ASIANPAINT.NS,2022-02-15,3143.45,10.59,8527.0,31.61,27859.0,99.44,6.21,33.93,7096.0,92.04,3122.90,-6.84,7.49,0.65,20.17,70.53,0.65
2,ASIANPAINT.NS,2022-05-16,3064.00,8.87,7893.0,31.60,29101.0,96.96,10.59,31.61,8527.0,99.44,3143.45,-0.03,-2.49,-2.52,-7.44,-16.24,-2.52
3,ASIANPAINT.NS,2022-08-15,3427.85,10.60,8607.0,36.27,32123.0,94.51,8.87,31.60,7893.0,96.96,3064.00,14.78,-2.90,11.88,9.05,19.50,11.88
4,ASIANPAINT.NS,2022-11-15,3053.20,8.16,8458.0,38.22,33485.0,79.88,10.60,36.27,8607.0,94.51,3427.85,5.38,-16.31,-10.93,-1.73,-23.02,-10.93
5,ASIANPAINT.NS,2023-02-15,2804.60,11.18,8637.0,38.81,33595.0,72.26,8.16,38.22,8458.0,79.88,3053.20,1.54,-9.69,-8.15,2.12,37.01,-8.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,ZEEL.NS,2021-11-15,312.70,2.81,1979.0,12.07,8449.0,25.91,2.23,10.24,1775.0,17.90,183.25,17.87,52.76,70.63,11.49,26.01,70.63
446,ZEEL.NS,2022-02-15,249.35,3.11,2113.0,11.02,7833.0,22.63,2.81,12.07,1979.0,25.91,312.70,-8.70,-11.56,-20.26,6.77,10.68,-20.26
447,ZEEL.NS,2022-05-16,239.25,1.89,2323.0,10.04,8190.0,23.83,3.11,11.02,2113.0,22.63,249.35,-8.89,4.83,-4.06,9.94,-39.23,-4.06
448,ZEEL.NS,2022-08-15,241.80,1.11,1846.0,8.92,8261.0,27.11,1.89,10.04,2323.0,23.83,239.25,-11.16,12.23,1.07,-20.53,-41.27,1.07


In [9]:
df_growth[(df_growth['Sales%'] > 0) & (df_growth['EPS%'] > 0)]['Stock'].value_counts()[0:20]

EICHERMOT.NS     5
FEDERALBNK.NS    5
BAJFINANCE.NS    5
LT.NS            4
IRCTC.NS         4
HINDUNILVR.NS    4
HCLTECH.NS       4
AUBANK.NS        4
CANBK.NS         4
ICICIBANK.NS     4
TCS.NS           4
TECHM.NS         4
WIPRO.NS         4
AXISBANK.NS      4
TATAPOWER.NS     4
INDUSINDBK.NS    3
INFY.NS          3
INDIANB.NS       3
ITC.NS           3
ASIANPAINT.NS    3
Name: Stock, dtype: int64

In [10]:
df_tmp_final = pd.DataFrame()
for stock in df_growth['Stock'].unique():
    df_tmp = df_growth[df_growth['Stock'] == stock][cols]
    df_tmp['sales_flag'] = 0
    df_tmp['eps_flag'] = 0
    df_tmp.loc[df_tmp['Sales%'] < 0, 'sales_flag'] = 1
    df_tmp.loc[df_tmp['EPS%'] < 0, 'eps_flag'] = 1
    df_tmp['sales_weight'] =  df_tmp['sales_flag']*5 + df_tmp['sales_flag'].shift(1)*4 + df_tmp['sales_flag'].shift(2)*3+df_tmp['sales_flag'].shift(3)*2+df_tmp['sales_flag'].shift(4)*1
    df_tmp['eps_weight'] =  df_tmp['eps_flag']*5 + df_tmp['eps_flag'].shift(1)*4 + df_tmp['eps_flag'].shift(2)*3+df_tmp['eps_flag'].shift(3)*2+df_tmp['eps_flag'].shift(4)*1
    df_tmp['total_weight'] = df_tmp['sales_weight'] + df_tmp['eps_weight'] 
    df_tmp = df_tmp.tail(1)
    df_tmp_final = df_tmp_final.append(df_tmp)

In [11]:
df_tmp_final[(df_tmp_final['Sales%'] > 0) & (df_tmp_final['EPS%'] > 0)].sort_values(by = 'total_weight', ascending=True)

,Stock,Quarter,Sales%,EPS%,Return%,sales_flag,eps_flag,sales_weight,eps_weight,total_weight
23,BAJFINANCE.NS,2023-02-15,12.79,6.92,-8.41,0,0,0.0,0.0,0.0
113,EICHERMOT.NS,2022-11-15,3.59,7.57,9.55,0,0,0.0,0.0,0.0
125,FEDERALBNK.NS,2023-02-15,10.05,14.66,-3.35,0,0,0.0,0.0,0.0
65,CANBK.NS,2023-02-15,7.42,12.14,-3.34,0,0,1.0,0.0,1.0
149,HCLTECH.NS,2023-02-15,8.16,17.34,1.54,0,0,0.0,3.0,3.0
11,AUBANK.NS,2023-02-15,6.16,14.37,2.18,0,0,0.0,3.0,3.0
17,AXISBANK.NS,2023-02-15,9.30,10.13,1.91,0,0,0.0,3.0,3.0
407,TECHM.NS,2023-02-15,4.61,0.83,-4.50,0,0,0.0,3.0,3.0
221,INDUSINDBK.NS,2023-02-15,12.62,8.72,0.73,0,0,3.0,0.0,3.0
443,WIPRO.NS,2023-02-15,3.06,14.64,2.30,0,0,0.0,3.0,3.0


In [12]:
# star stocks
df_tmp_final[df_tmp_final['total_weight'] <= 6].sort_values(by = 'total_weight', ascending=True)['Stock'].values

array(['BAJFINANCE.NS', 'EICHERMOT.NS', 'FEDERALBNK.NS', 'CANBK.NS',
       'AUBANK.NS', 'TCS.NS', 'INDUSINDBK.NS', 'TECHM.NS', 'ICICIBANK.NS',
       'AXISBANK.NS', 'HCLTECH.NS', 'WIPRO.NS', 'TATAPOWER.NS', 'HDFC.NS',
       'HDFCBANK.NS', 'HINDUNILVR.NS', 'EMAMILTD.NS', 'INDIANB.NS',
       'BANKBARODA.NS', 'INFY.NS', 'KOTAKBANK.NS', 'SBIN.NS',
       'TATACONSUM.NS', 'ITC.NS', 'LT.NS'], dtype=object)

In [13]:
scrip = 'HDFCBANK.NS'
df_growth[df_growth['Stock'] == scrip][cols]

,Stock,Quarter,Sales%,EPS%,Return%
157,HDFCBANK.NS,2022-02-15,2.14,16.31,-5.13
158,HDFCBANK.NS,2022-05-16,3.42,-1.47,-12.34
159,HDFCBANK.NS,2022-08-15,7.43,-8.39,15.01
160,HDFCBANK.NS,2022-11-15,11.94,15.77,8.74
161,HDFCBANK.NS,2023-02-15,11.98,14.02,2.58
